# The Cournot model and the possibility of collusion

## The model

In [1]:
import numpy as np
%matplotlib inline
import pandas as pd
from scipy import optimize, arange
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In this project we want to investigate the cournot equilibrium and in this regard investigate when collusion can be optained. As an extra feature we check this result for different degrees of elasticity of substitution between the goods (i.e when the firm's goods are seen as similar of different degrees).

For simplicity we will in this project work with only two firms indicating a duopoly but the project can easily be extended to include more firms in the market.
We start by defining a linear demand curve for good $x_i$ given as: \\[p(x_i,x_{j},b)=100-x_i-bx_j\\] where b indicates the degree of substitution between the goods (if b=1 we have perfect substitutes and if b<1 the goods are seen as different).

Now having the demand, we need to specify the cost and we assume that the firms incurs fixed marginal costs and the cost function is given by: \\[c(x,f)=cx+F\\] where F is fixed costs. The profit for firm i is given by: \\[\pi_i(x_i,x_{j},c,b)=p(x_i,x_{j},b) \cdot x_i-c_i(x_i,F)\\] unless the firms incurs a negative profit, then it will not produce (we investige the production in the long run) which implies the following demand, cost functions and profit functions. 

In [2]:
def demand(xi,xj,b):
    return 100-xi-b*xj
def cost(xi,ci,F):
    if ci==0:
        cost=0
    else:
        cost=ci*xi+F
    return cost
def profit(xi,xj,ci,b,F):
    return demand(xi,xj,b)*xi-cost(xi,ci,F)

## Reaction functions

Unlike the Bertrand model where firms compete by setting prices, in the cournot model, the firms maximize profits by choosing the amount to be produced and the price are then determined by this quantity in the demand function. Each firm then maximize their profit function given the production of the other and $x_i^*,x_j^*$ indicate the nash equilibrium indicate the point where neither of the firms can do better by deviating from this point. This is given by: \\[x_i^*= \underset{x_i} \max \, \pi(x_i,x_j^*,c,b) \\] and $x_j$ is found in the same way for $x_i^*$

This optimal output given the production of the other firm is called the reaction function and it is found using the first order derivative (as $\pi_i$ is clearly concave wrt $x_i$):
\\[ \frac{\partial \pi (x_i,x_j^*)}{\partial x_i} =0 \\]
and then solving for $x_i$. The reaction function can be found analytically to be: \\[ x_i^*=\frac{1}{2}\left[ 100-bx_j-c_i\right] \\]
But can also be found by numerical optimization using the scipy optimizer. Initially we use the brute method which evaluates the profit function at each point of prespecified range. A natural choice of this range is 0 to 100 given the specified demand function. The pros for this method is that it is that it requires very little information and there is no need for any derivatives. The optimizer simply evaluates the function at different values and find the minimum value. Naturally the con of this approach is that it very inefficient and the number of derivation increase exponentially as the range/grid increases.

In [6]:
def reac_func(xj,ci,b,F):
    xi = optimize.brute(lambda x: -profit(x,xj,ci,b,F), ((0,100),),) #As brute is a minizer -profit is the optimal profit value
    return xi[0] #Returns the xi value xi[1] would provide the actual profit for this production

We can now draw the reaction function for a given values of $c_i,b,F$ using the ipywi package. As pyplot take values and not functions as input we find the reaction function for a range of $x_j$.

In [7]:
from pylab import *
def draw_rf(ci=10,b=1,F=0): #We need to input of ci and b and F to make draw the reaction function. We can make these changeable with the ipywidget package
    
    xj_range=np.arange(0,(100-ci)/b,0.5) #This is the values for which xi is between producing 0 and producing for the whole market.
    i=0
    xi_range=np.empty(len(xj_range))
    
    while i < len(xj_range):
        xi_range[i]= reac_func(xj_range[i],ci,b,F)
        i=i+1
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(xj_range, xi_range)
    
    upper_xj=(100-ci)/b
    ax.set_xlim(0, upper_xj)
    
    upper_xi=0.5*(100-ci)
    ax.set_ylim(0, upper_xi)
    
    plt.xlabel('xj')
    plt.ylabel('xi')
    plt.show()

In [9]:
#interactive_plot=interact(draw_rf, ci=widgets.IntSlider(min=0,max=40,step=1,value=10), b=0.5, F=fixed(0));
#interactive_plot=interact(draw_rf, ci=10, b=0.5);
#interactive_plot = interactive(draw_rf, ci=10, b=0.5);
#interactive_plot
#Ide til forbedring her: Sørg for b ikke kan gå under 0.10 eftersom den bliver tom eller en fejl :D 
interact(draw_rf, ci=10, b=0.5);

interactive(children=(IntSlider(value=10, description='ci', max=30, min=-10), FloatSlider(value=0.5, descripti…

We can find the reaction function for the other firm j as well in a similar manner. Having obtained the reaction function for firm i and firm j, we can find the nash equilibrium as the solution to the two equations with two unknows:

\\[ x_i^*=\frac{1}{2}\left[ 100-bx_j-c_i\right] \\] \\[ x_j^*=\frac{1}{2}\left[ 100-bx_i-c_j\right] \\] which has an analytical solution that we now want to find using one of scipy's optimizers. 

The general idea is that we want to solve: \\[ \begin{bmatrix}
    x_i^*  \\
    x_j^*
\end{bmatrix} = \begin{bmatrix}
    r(x_j^*)  \\
    r(x_i^*)
\end{bmatrix} \leftrightarrow \begin{bmatrix}
    x_i^*-r(x_j^*)  \\
    x_j^*-r(x_i^*)
\end{bmatrix} = \begin{bmatrix}
    0  \\
    0
\end{bmatrix}\\]
Where $r(x_j),r(x_i)$ is the reaction function of $x_i, x_j$ respectively. We are going to use fsolve to solve this system as it essentially finds the root of the above.

In [11]:
def vec_reac(x,ci=10,cj=10,b=1,F=0):
    return array(x)-array([reac_func(x[1],ci,b,F),reac_func(x[0],cj,b,F)])

We now need to come with an initial guess and using scipys fsolve which solves the equation \\[ \begin{bmatrix}
    x_i^*-r(x_j^*)  \\
    x_j^*-r(x_i^*)
\end{bmatrix} = \begin{bmatrix}
    0  \\
    0
\end{bmatrix}\\]

In [12]:
VEC=[30,30]
parameters=(10,10,1,0) #parameters are ci,cj,b,F
nash_eq=optimize.fsolve(vec_reac, VEC, args = (parameters))
print(nash_eq)

[30. 30.]


!!! Ovenstående del kunne man evt. visualisere på en pæn måde, hvis i vil lave det? !!!

The cournot equilibrium is based on the assumption that the firms produce in order to maximize their profit in the given production period given the action of the other firm. But what if the firms meet each other several times and have the choice of producing continously period after period. Can the firms do better for themselves by coluding? 

Some very bright microeconomics have realized that it would be optimal for firm i to collude/collaborate if \\[  \pi(x_i^c,x_j^c)\frac{1}{1-\delta}\geq \underset{x_i} \max (x_i,x_j^c)+ \frac{\delta}{1-\delta} \pi(x_i^*,x_j^*)\\]
Where $x_i^c$ is the collution amount produced by firm i and $\delta$ is the discount rate between periods. $x_i^*$ is still the cournot nash equilibrium. For simplicity we will assume in the following that the firms have the same coststructure and $c_i=c_j$ which will result in a symmetric solution.

In [13]:
VEC=[30,30]
def col_output(x,b=1,c=10,delta=0.05,F=0):
    profit_ij=profit(x,x,c,b,F)
    nash_eq=optimize.fsolve(vec_reac, VEC, args = ([c,c,b,F]))
    if profit_ij >= (1-delta)*profit(reac_func(x,c,b,F),x,c,b,F)+delta*profit(nash_eq[0],nash_eq[1],c,b,F):
        collusion_profit = 2*profits # There is collusion
    else:
        collusion_profit = 0 # Collusion can not be obtained
    return industry_profits
#remeber the functions:
#def reac_func(xj,ci,b,F)
#def profit(xi,xj,ci,b,F):

When there is no chance of collusion we set the result to zero in order to illustrate that the result will be the nash equilibrium and not the collusion output. This will be clear in the following code:

In [24]:
range_x = arange(25,nash_eq[0],0.1) #start, end and step size
range_prof = [col_output(x,1,10,0.05,0) for x in range_x]
plt.clf()
plt.plot(range_x, range_prof,'-', color = 'b', linewidth = 2.5)
plt.show()

ValueError: setting an array element with a sequence.

In [15]:
range_x = arange(25,nash_eq[0],0.1)

In [16]:
range_x

array([25. , 25.1, 25.2, 25.3, 25.4, 25.5, 25.6, 25.7, 25.8, 25.9, 26. ,
       26.1, 26.2, 26.3, 26.4, 26.5, 26.6, 26.7, 26.8, 26.9, 27. , 27.1,
       27.2, 27.3, 27.4, 27.5, 27.6, 27.7, 27.8, 27.9, 28. , 28.1, 28.2,
       28.3, 28.4, 28.5, 28.6, 28.7, 28.8, 28.9, 29. , 29.1, 29.2, 29.3,
       29.4, 29.5, 29.6, 29.7, 29.8, 29.9])